# Conexiones módulo PWM UC3844
[Datasheet UC3844](../Datasheets/PWM_UC3844.pdf)

## Frecuencia
Cálculo de $C_T$ y $R_T$ para fijar $f_{sw}$

$$f_{sw} = \frac{1.72}{C_T \cdot R_T}$$

Queremos $f_{sw} = 150 kHz$

In [6]:
from utils import C_values, R_values, to_unit, parallel

fsw = 150E3
Rd = 0

tol = 0.1
CT = []
RT = []

for Ct in C_values:
    if 1E-9 < Ct < 100E-9:
        for Rt in R_values:
            if 5E3 < Rt < 100E3: 
                f = 1.72/(Ct*Rt)
                if fsw*(1-tol) < f < fsw*(1+tol):
                    CT.append(Ct)
                    RT.append(Rt)
        
print(f'Se encontraron {len(CT)} opciones')
for i in range(len(CT)):
    print('CT = ' + to_unit(CT[i]) + 'F con RT = ' + to_unit(RT[i]) + 'Ohm, resultando en fsw = ' + to_unit(1.72/(CT[i]*RT[i]), 5) + 'Hz')

if len(CT) == 0:
    print('Buscando con resistencias en paralelo\n')

    Rpar = []
    for Ct in C_values:
        if 1E-9 < Ct < 200E-9:
            i = 0
            j = 0
            for i in range(len(R_values)):
                for j in range(i, len(R_values)):
                    Rt = parallel(R_values[i], R_values[j])
                    if 5E3 < Rt < 100E3: 
                        f = 1.72/(Ct*Rt)
                        if fsw*(1-tol) < f < fsw*(1+tol):
                            CT.append(Ct)
                            RT.append((R_values[i], R_values[j]))

    print(f'Se encontraron {len(CT)} opciones')
    for i in range(len(CT)):
        print('CT = ' + to_unit(CT[i]) + 'F con RT1 = ' + to_unit(RT[i][0]) + 'Ohm y RT2 = ' + to_unit(RT[i][1]) + 'Ohm => RT = ' + to_unit(parallel(RT[i][0], RT[i][1])) + 'Ohm, resultando en fsw = ' + to_unit(1.72/(CT[i]*parallel(RT[i][0], RT[i][1])), 5) + 'Hz')
        print('Rango: RT1 = 5.6KOhm -> ' + 'fsw = ' + to_unit(1.72/(CT[i]*parallel(5600, RT[i][1])), 5) + 'Hz | RT1 = 10.6kOhm -> ' + 'fsw = ' + to_unit(1.72/(CT[i]*parallel(10.6E3, RT[i][1])), 5) + 'Hz\n')

Se encontraron 4 opciones
CT = 1.5nF con RT = 8.2kOhm, resultando en fsw = 139.84kHz
CT = 2nF con RT = 5.6kOhm, resultando en fsw = 153.57kHz
CT = 2.2nF con RT = 5.1kOhm, resultando en fsw = 153.3kHz
CT = 2.2nF con RT = 5.6kOhm, resultando en fsw = 139.61kHz


Podemos elegir cualquiera entre:

CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 100kOhm => RT = 7.58kOhm, resultando en fsw = 151.3kHz

	Rango: RT1 = 5.6KOhm -> fsw = 216.23kHz | RT1 = 10.6kOhm -> fsw = 119.64kHz

CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 120kOhm => RT = 7.68kOhm, resultando en fsw = 149.39kHz

	Rango: RT1 = 5.6KOhm -> fsw = 214.32kHz | RT1 = 10.6kOhm -> fsw = 117.73kHz

CT = 2nF con RT1 = 6.8kOhm y RT2 = 39kOhm => RT = 5.79kOhm, resultando en fsw = 148.52kHz

	Rango: RT1 = 5.6KOhm -> fsw = 175.62kHz | RT1 = 10.6kOhm -> fsw = 103.18kHz

CT = 2.2nF con RT1 = 6.8kOhm y RT2 = 22kOhm => RT = 5.19kOhm, resultando en fsw = 150.51kHz

	Rango: RT1 = 5.6KOhm -> fsw = 175.15kHz | RT1 = 10.6kOhm -> fsw = 109.29kHz

Al final pusimos $C_T = 1.5nF $ y $R_T = 5.6KOhm $ + 1 PRESET de $5k \Omega$ ajustable

$\Rightarrow f_{sw} = 150kHz$

## Divisor de tensión medida
$\overline{V_o} = 112.5V$

$V_{comp} = 2.5V$

$V_{o_{comp}} = \frac{R_a}{R_a + R_b} \cdot \overline{V_o} \approx V_{comp}$

In [40]:
from utils import R_values, to_unit

Vcomp = 2.5
Vo = 112.5

tol = 0.01
Ra = []
Rb = []

for ra in R_values:
	if 100 < ra < 150E3: 
		for rb in R_values:
			if 100 < rb < 150E3: 
				Vo_comp = Vo*ra/(ra + rb)
				if Vcomp*(1-tol) < Vo_comp < Vcomp*(1+tol):
					Ra.append(ra)
					Rb.append(rb)
        
print(f'Se encontraron {len(Ra)} opciones')
for i in range(len(Ra)):
    Ra[i] = 1860
    Rb[i] = 82000
    print('Ra = ' + to_unit(Ra[i]) + 'Ohm con Rb = ' + to_unit(Rb[i]) + 'Ohm, resultando en Vo_comp = ' + to_unit(Vo*Ra[i]/(Ra[i] + Rb[i]), 5) + 'V')
    print('Rango:\tRa = 1.8kOhm -> Vo_comp = ' + to_unit(Vo*1800/(1800 + Rb[i]), 5) + 'V')
    print('\tRa = 1.9kOhm -> Vo_comp = ' + to_unit(Vo*1910/(1910 + Rb[i]), 5) + 'V')
    
print('Queremos: 2.43V < Vo_comp < 2.56V')

Se encontraron 2 opciones
Ra = 1.86kOhm con Rb = 82kOhm, resultando en Vo_comp = 2.4952V
Rango:	Ra = 1.8kOhm -> Vo_comp = 2.4165V
	Ra = 1.9kOhm -> Vo_comp = 2.5608V
Ra = 1.86kOhm con Rb = 82kOhm, resultando en Vo_comp = 2.4952V
Rango:	Ra = 1.8kOhm -> Vo_comp = 2.4165V
	Ra = 1.9kOhm -> Vo_comp = 2.5608V
Queremos: 2.43V < Vo_comp < 2.56V


Tomamos $R_a = 1.8k\Omega$ + Pote $100\Omega$ y $R_b = 82k\Omega$

Rango:	
	
$R_a = 1.8k\Omega \rightarrow V_{o_{comp}} = 2.4165V$
	
$R_a = 1.91k\Omega \rightarrow V_{o_{comp}} = 2.5608V$

## Compensación de rampa

Inciso 9.2.2.10.2 de la datasheet (Páginas 28 y 29)

In [16]:
import numpy as np
from utils import to_unit

# Datos
fsw = 150E3
D = 0.6
Vdmin = 25
Rcs = 0.1
L1 = 588.11E-6
Vosc = 1.7

Mideal = (1/np.pi + 0.5)/(1 - D)
print(f'Mideal = {Mideal:.4g}')

# Pendiente de In1
Sn = Vdmin*Rcs/L1
Snp = Sn*1E-6
print(f'Sn = {Snp:.4g} V/us')

# Pendiente de compensación
Se = (Mideal - 1)*Sn
Sep = Se*1E-3
print(f'Se = {Sep:.4g} mV/us')

# Pendiente del oscilador
ton = D/fsw
Sosc = Vosc/ton
Soscp = Sosc * 1E-3
print(f'Sosc = {Soscp:.4g} mV/us')

# Resistencia de rampa
Rcsf = 1E3
Cramp = 10E-9
Rramp = Rcsf * (Sosc/Se - 1)
print(f'Rramp = ' + to_unit(Rramp) + 'Ohm')



Mideal = 2.046
Sn = 0.004251 V/us
Se = 4.445 mV/us
Sosc = 425 mV/us
Rramp = 94.6kOhm


Tomamos $R_{ramp}  =100k\Omega$ y $C_{ramp} = 10nF$ por ahora

## Compensación por realimentación

Como no tenemos la transferencia todavía no la hicimos